In [12]:
import json
import requests

In [13]:
import geopandas as gpd
import pandas as pd

In [14]:
from tqdm.notebook import tqdm

## Open State data

In [15]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

## Get Poplation Data

In [16]:
state_populations = pd.read_excel(
    "data/NST-EST2024-POP.xlsx", sheet_name=None, engine="openpyxl"
)

In [17]:
state_populations_df = state_populations["NST-EST2024-POP"][
    [
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)",
        "Unnamed: 5",
    ]
]
state_populations_df = state_populations_df.rename(
    columns={
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)": "NAME",
        "Unnamed: 5": "POPULATION",
    }
)
state_populations_df["NAME"] = state_populations_df["NAME"].str[1:]

In [18]:
states_with_population_df = states_df.merge(state_populations_df, on="NAME", how="left")
states_with_population_df = states_with_population_df[
    ["STUSPS", "NAME", "POPULATION", "geometry"]
]

In [19]:
states_with_population_df = states_with_population_df.dropna()

## Get Police Data

In [5]:
url = "https://data.bls.gov/OESServices/resultsgeoocc"
headers = {
    "accept": "application/json, text/plain, */*",
    "accept-language": "en-US,en;q=0.9,ru;q=0.8,el;q=0.7",
    "cache-control": "no-cache",
    "content-type": "application/json;charset=UTF-8",
    "pragma": "no-cache",
    "sec-ch-ua": '"Google Chrome";v="141", "Not?A_Brand";v="8", "Chromium";v="141"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "cookie": "_ga=GA1.1.1422392323.1761498607; nmstat=87e614f7-860e-eda0-1961-b768847a4d81; _ga_CSLL4ZEK4L=GS2.1.s1761498607$o1$g1$t1761498779$j59$l0$h0",
    "Referer": "https://data.bls.gov/oes/",
}

In [6]:
body = {
    "areaTypeCode": "S",
    "areaCode": "9900000",
    "industryCode": "000000",
    "occupationCode": ["333051"],
    "datatype": ["01"],
    "releaseDateCode": ["2024A01", "2024A01"],
    "outputType": "H",
}
r = requests.post(url, headers=headers, data=json.dumps(body))

In [8]:
data = []
for i in tqdm(range(len(states_df)), desc="Parsing States"):
    body = {
        "areaTypeCode": "S",
        "areaCode": f"{states_df.iloc[i]['STATEFP']}00000",
        "industryCode": "000000",
        "occupationCode": ["333051"],
        "datatype": ["01"],
        "releaseDateCode": ["2024A01", "2024A01"],
        "outputType": "H",
    }
    r = requests.post(url, headers=headers, data=json.dumps(body))
    json_resp = r.json()
    if json_resp["resultsGeoOccVO"][0]["occupations"]:
        police_officers = int(
            r.json()["resultsGeoOccVO"][0]["occupations"][0]["values"][0]["value"]
        )
        data.append(
            {"STUSPS": states_df.iloc[i]["STUSPS"], "officers": police_officers}
        )
    else:
        print(states_df.iloc[i]["NAME"])

Parsing States:   0%|          | 0/56 [00:00<?, ?it/s]

American Samoa
United States Virgin Islands
Guam
Commonwealth of the Northern Mariana Islands


In [20]:
occupation_df = pd.DataFrame(
    data,
)

## Merge Data

In [21]:
occupation_gdf = states_with_population_df.merge(
    occupation_df, on="STUSPS", how="inner"
).dropna()

In [27]:
occupation_gdf["per_100k"] = (
    (occupation_gdf["officers"] / (occupation_gdf["POPULATION"] / 100_000))
    .round(decimals=0)
    .astype(int)
)
occupation_gdf["per_1m"] = (
    (occupation_gdf["officers"] / (occupation_gdf["POPULATION"] / 1_000_000))
    .round(decimals=0)
    .astype(int)
)

In [29]:
occupation_gdf = occupation_gdf.to_crs(9311)
occupation_gdf.to_file("data/Police_Officers_Per_State.gpkg")